<a href="https://colab.research.google.com/github/eirikngard/Eika/blob/master/Vannskade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vise vannskadedata fra Innsikt for ulike skadeårsaker 

## Importere nødvendige biblioteker


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Lese filer lagret lokalt

In [151]:
from google.colab import files
uploaded = files.upload()

Saving naturskade_ikke_pool.xlsx to naturskade_ikke_pool.xlsx
Saving vannskade_frost.xlsx to vannskade_frost (2).xlsx
Saving vannskade_tilstopping.xlsx to vannskade_tilstopping (2).xlsx
Saving vannskade_vanninntregning.xlsx to vannskade_vanninntregning (1).xlsx


## Enkel lineær regresjon

In [10]:
def linreg(X, Y):
    """
    return a,b in solution to y = ax + b such that root mean square distance between trend line and original points is minimized
    """
    N = len(X)
    Sx = Sy = Sxx = Syy = Sxy = 0.0
    for x, y in zip(X, Y):
        Sx = Sx + x
        Sy = Sy + y
        Sxx = Sxx + x*x
        Syy = Syy + y*y
        Sxy = Sxy + x*y
    det = Sxx * N - Sx * Sx
    return (Sxy * N - Sy * Sx)/det, (Sxx * Sy - Sx * Sxy)/det



## Hente data fra importerte filer

In [11]:
navn = ['år','mnd','skade']
år = np.arange(2000,2030,1)

frost = pd.read_excel('vannskade_frost.xlsx',header=None,skiprows=3,names=navn)
tilstopp = pd.read_excel('vannskade_tilstopping.xlsx',header=None,skiprows=3,names=navn)
intre = pd.read_excel('vannskade_vanninntregning.xlsx',header=None,skiprows=3,names=navn)
nat = pd.read_excel('naturskade_ikke_pool.xlsx',header=None,skiprows=3,names=navn)

data_frost = []
data_til = []
data_inn = []
data_nat = []
for i in år[:-9]:
  frost1 = frost.loc[frost['år']==i,'skade'].sum()
  tilstopp1 = tilstopp.loc[tilstopp['år']==i,'skade'].sum()
  intre1 = intre.loc[intre['år']==i,'skade'].sum()
  nat1 = nat.loc[nat['år']==i,'skade'].sum()
  data_frost.append(frost1)
  data_til.append(tilstopp1)
  data_inn.append(intre1)
  data_nat.append(nat1)

## Plotte data

In [12]:
data_alle = np.array([data_frost,data_til,data_inn,data_nat])#.astype(np.float)
tit = np.array(['Vannskade Frost','Vannskade Tilstopp','Vannskade Inntrengning',
                'Naturaskade Alle'])
grid = np.array([0,1,2,3])
#fig, ax = plt.subplots()#, figsize=(10,10))
bins = np.linspace(0,50000000,10000000)
for i,j in zip(data_alle,tit):

  snitt = np.mean(i)
  a,b = linreg(range(len(i)),i)  #//x=skade
  extrapolatedtrendline=[a*index + b for index in range(len(år))]
  plt.plot(år,extrapolatedtrendline,label="trend")
  plt.plot(år[:-9],i,label="skade [kr]")
  plt.axhline(y=snitt,color='r',label='snitt')
  plt.title("Utbetalt,"+str(j))#str(i))
  plt.ylabel("mill")
  #plt.yticks(bins)
  plt.grid(),plt.legend(), 
  plt.show()


Error in callback <function flush_figures at 0x7f5436e01ea0> (for post_execute):


KeyboardInterrupt: ignored

In [7]:
import numpy as np
bin = np.linspace(0,50000000,10000000)
bin

array([0.0000000e+00, 5.0000005e+00, 1.0000001e+01, ..., 4.9999990e+07,
       4.9999995e+07, 5.0000000e+07])